In [ ]:
from dspy.datasets import HotPotQA
import dspy

## Load the dataset

In [ ]:
dataset = HotPotQA(train_seed=1, test_size=0, train_size=1000)
# Tell DSPy that the 'question' field is the input. Any other fields are labels and/or metadata.
dataset = [x.with_inputs('question') for x in dataset.train]
print(len(dataset))

In [ ]:
from dspy.retrieve.qdrant_rm import QdrantRM
from qdrant_client import QdrantClient

In [ ]:
qdrant_client = QdrantClient(":memory:")  # In-memory load
docs = [x.question + " -> " + x.answer for x in dataset]
ids = list(range(0,len(docs)))

In [ ]:
qdrant_client.add(
    collection_name="hotpotqa",
    documents=docs,
    ids=ids
    )

In [ ]:
qdrant_retriever_model = QdrantRM("hotpotqa", qdrant_client, k=3)

In [ ]:
def get_top_passages(question):
    retrieve = dspy.Retrieve(k=3)
    topK_passages = retrieve(question, k=3).passages
    print(f"Top {retrieve.k} passages for question: {question} \n", '-' * 30, '\n')
    for idx, passage in enumerate(topK_passages):
        print(f'{idx+1}]', passage, '\n')

In [ ]:
dev_example = dataset[100]

get_top_passages(dev_example.question)